In [13]:
# Load packages:
import numpy as np
import pandas as pd
import re

pd.set_option('display.max_rows', None)

## Goal: make this csv usable to link information on criticality and degree with the dataset of alerts recorded
## Table of content:

- encoding "critico" and "colore":
    - "critico" (binary variable 0,1)
    - "colore" --> link the color of each pdo to the one of the related pdm (binary variable giallo,azzurro) [source: "cenni alla diagnostica"]
- recoding "codice": numerical code only and entire code
- change Critico=X to alerts 5-3, 5-3-00, 5-5, 5-5-00 [source: mail 06/05]
- change the code of PDO 00 to 99 [source: mail 24/05]
- give colour=azzurro to PDM from 5-31 to 5-99 [source: videocall Trenord 12/07 and mail 18/07]
- add fake PDM 5-20-0 and 5-21-0 [source: ricevimento]
- link PDMs' colours to every PDO
- clean it to obtain the final dataset

In [14]:
# Load the dataset:
data = pd.read_csv('Codici_diagnostici_TCU_critici.csv', delimiter=";")
# check data:
print(data.head())

  Critico Codice gruppo   Codice sottogruppo  1 Codice sottogruppo  2  Colore
0       X         02 AT  19 Interv. Rele' Diff.                   NaN  Giallo
1       X         02 AT  19 Interv. Rele' Diff.     01 KD1 prolungato  Grigio
2       X         02 AT  19 Interv. Rele' Diff.       02 KD1 ripetuto  Grigio
3       X        05 TCU           01 Avaria TCU                   NaN  Giallo
4       X        05 TCU           01 Avaria TCU         01 Stotz 4F01  Grigio


In [15]:
# Copy the dataset to modify it:
df = data.copy()
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Critico                22 non-null     object
 1   Codice gruppo          246 non-null    object
 2   Codice sottogruppo  1  246 non-null    object
 3   Codice sottogruppo  2  219 non-null    object
 4   Colore                 237 non-null    object
dtypes: object(5)
memory usage: 9.7+ KB


,Critico,Codice gruppo,Codice sottogruppo 1,Codice sottogruppo 2,Colore
0,X,02 AT,19 Interv. Rele' Diff.,NaN,Giallo
1,X,02 AT,19 Interv. Rele' Diff.,01 KD1 prolungato,Grigio
2,X,02 AT,19 Interv. Rele' Diff.,02 KD1 ripetuto,Grigio
3,X,05 TCU,01 Avaria TCU,NaN,Giallo
4,X,05 TCU,01 Avaria TCU,01 Stotz 4F01,Grigio
5,NaN,05 TCU,01 Avaria TCU,02 Life MVB TCU,Grigio
6,NaN,05 TCU,03 Blocco Perm CA,NaN,Giallo
7,NaN,05 TCU,03 Blocco Perm CA,00 Blocco Perm CA,Grigio
8,NaN,05 TCU,03 Blocco Perm CA,01 Blocco Temp CA,Grigio
9,NaN,05 TCU,03 Blocco Perm CA,02 IncongruenzaK4,Grigio


In [16]:
# Add critico=X to alerts 5-3, 5-3-00, 5-5, 5-5-00:
#print(df.loc[0, 'Critico'])
#print(type(df.loc[0, 'Critico']))
df['Critico'] = df.apply(lambda row: 'X' if row.name in {6, 7, 40, 41} else row['Critico'], axis=1)
df

,Critico,Codice gruppo,Codice sottogruppo 1,Codice sottogruppo 2,Colore
0,X,02 AT,19 Interv. Rele' Diff.,NaN,Giallo
1,X,02 AT,19 Interv. Rele' Diff.,01 KD1 prolungato,Grigio
2,X,02 AT,19 Interv. Rele' Diff.,02 KD1 ripetuto,Grigio
3,X,05 TCU,01 Avaria TCU,NaN,Giallo
4,X,05 TCU,01 Avaria TCU,01 Stotz 4F01,Grigio
5,NaN,05 TCU,01 Avaria TCU,02 Life MVB TCU,Grigio
6,X,05 TCU,03 Blocco Perm CA,NaN,Giallo
7,X,05 TCU,03 Blocco Perm CA,00 Blocco Perm CA,Grigio
8,NaN,05 TCU,03 Blocco Perm CA,01 Blocco Temp CA,Grigio
9,NaN,05 TCU,03 Blocco Perm CA,02 IncongruenzaK4,Grigio


In [17]:
# Add fake PDM 5-20 and 5-21 in order to aggregate the dataset according to a priori criterion based on PDM:
df.loc[len(df)] = {'Critico': np.nan, 'Codice gruppo': '05 TCU', 'Codice sottogruppo  1': '20', 'Codice sottogruppo  2': np.nan, 'Colore': 'Azzurro'}
df.loc[len(df)] = {'Critico': np.nan, 'Codice gruppo': '05 TCU', 'Codice sottogruppo  1': '21', 'Codice sottogruppo  2': np.nan, 'Colore': 'Azzurro'}
df

,Critico,Codice gruppo,Codice sottogruppo 1,Codice sottogruppo 2,Colore
0,X,02 AT,19 Interv. Rele' Diff.,NaN,Giallo
1,X,02 AT,19 Interv. Rele' Diff.,01 KD1 prolungato,Grigio
2,X,02 AT,19 Interv. Rele' Diff.,02 KD1 ripetuto,Grigio
3,X,05 TCU,01 Avaria TCU,NaN,Giallo
4,X,05 TCU,01 Avaria TCU,01 Stotz 4F01,Grigio
5,NaN,05 TCU,01 Avaria TCU,02 Life MVB TCU,Grigio
6,X,05 TCU,03 Blocco Perm CA,NaN,Giallo
7,X,05 TCU,03 Blocco Perm CA,00 Blocco Perm CA,Grigio
8,NaN,05 TCU,03 Blocco Perm CA,01 Blocco Temp CA,Grigio
9,NaN,05 TCU,03 Blocco Perm CA,02 IncongruenzaK4,Grigio


In [19]:
# Fill NaN with 0s:
missing_values_count = df.isnull().sum()
print(missing_values_count)

df["Critico_0"] = df["Critico"].fillna(0)
df["Codice_sottogruppo_2_0"] = df["Codice sottogruppo  2"].fillna(0)

df

Critico                   222
Codice gruppo               0
Codice sottogruppo  1       0
Codice sottogruppo  2      29
Colore                      9
Critico_0                   0
Codice_sottogruppo_2_0      0
dtype: int64


,Critico,Codice gruppo,Codice sottogruppo 1,Codice sottogruppo 2,Colore,Critico_0,Codice_sottogruppo_2_0
0,X,02 AT,19 Interv. Rele' Diff.,NaN,Giallo,X,0
1,X,02 AT,19 Interv. Rele' Diff.,01 KD1 prolungato,Grigio,X,01 KD1 prolungato
2,X,02 AT,19 Interv. Rele' Diff.,02 KD1 ripetuto,Grigio,X,02 KD1 ripetuto
3,X,05 TCU,01 Avaria TCU,NaN,Giallo,X,0
4,X,05 TCU,01 Avaria TCU,01 Stotz 4F01,Grigio,X,01 Stotz 4F01
5,NaN,05 TCU,01 Avaria TCU,02 Life MVB TCU,Grigio,0,02 Life MVB TCU
6,X,05 TCU,03 Blocco Perm CA,NaN,Giallo,X,0
7,X,05 TCU,03 Blocco Perm CA,00 Blocco Perm CA,Grigio,X,00 Blocco Perm CA
8,NaN,05 TCU,03 Blocco Perm CA,01 Blocco Temp CA,Grigio,0,01 Blocco Temp CA
9,NaN,05 TCU,03 Blocco Perm CA,02 IncongruenzaK4,Grigio,0,02 IncongruenzaK4


In [21]:
# Add, arbitrary, colore="azzurro" for PDM 5-31,5-32,.....,5-99:
index_fake_pdm = [94,111,128,145,163,181,199,207,235]
for i in index_fake_pdm:
    df.loc[i, 'Colore'] = 'Azzurro'

# check:
missing_values_count = df.isnull().sum()
print(missing_values_count)

Critico                   222
Codice gruppo               0
Codice sottogruppo  1       0
Codice sottogruppo  2      29
Colore                      0
Critico_0                   0
Codice_sottogruppo_2_0      0
dtype: int64


In [22]:
# Encoding "Critico" as a binary taking 0 and 1 (=X):
df['Critico_0'] = df['Critico_0'].str.lower()

df['Criticality'] = pd.Series([1 if x == "x" else 0 for x in df['Critico_0']]).astype('category')
print(df['Criticality'])

print("\n")
sum_by_group = df.groupby("Criticality").size()
print(sum_by_group)

# there are 26 alerts identified as critical and 220 not critical

0      1
1      1
2      1
3      1
4      1
5      0
6      1
7      1
8      0
9      0
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     0
18     0
19     1
20     0
21     0
22     1
23     0
24     0
25     0
26     0
27     0
28     0
29     1
30     0
31     0
32     0
33     0
34     0
35     0
36     0
37     0
38     0
39     1
40     1
41     1
42     0
43     0
44     0
45     0
46     0
47     1
48     1
49     1
50     0
51     0
52     0
53     0
54     0
55     0
56     0
57     0
58     0
59     0
60     0
61     0
62     1
63     1
64     1
65     0
66     0
67     0
68     0
69     0
70     0
71     0
72     0
73     0
74     0
75     0
76     0
77     0
78     0
79     0
80     0
81     0
82     0
83     0
84     0
85     0
86     0
87     0
88     0
89     0
90     0
91     0
92     0
93     0
94     0
95     0
96     0
97     0
98     0
99     0
100    0
101    0
102    0
103    0
104    0
105    0
106    0
107    0
108    0
109    0
110    0
1

/tmp/ipykernel_2441328/2986128187.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sum_by_group = df.groupby("Criticality").size()


In [23]:
# Encoding "Colore" as a category variable where the color of each pdo is the one of the pdm associated:
sum_by_group = df.groupby("Colore").size()
print(sum_by_group) # there are: 9 "Azzurro", 9 "Giallo", 76 "Grigio"

df['Colore_low'] = df['Colore'].str.lower()

for i in range(1, len(df)):
    if df.loc[i, "Colore_low"] == "grigio":
        df.loc[i, 'Colore_low'] = (df.loc[i-1, "Colore_low"])
    else:
        df.loc[i, 'Colore_low'] = (df.loc[i, "Colore_low"])
        
print(df['Colore_low'])

sum_by_group = df.groupby("Colore_low", dropna=False).size()
print(sum_by_group) # there are: 179 "Azzurro", 69 "Giallo"

df["Colour"] = df['Colore_low'].astype('category')
df

Colore
Azzurro     20
Giallo       9
Grigio     219
dtype: int64
0       giallo
1       giallo
2       giallo
3       giallo
4       giallo
5       giallo
6       giallo
7       giallo
8       giallo
9       giallo
10      giallo
11      giallo
12      giallo
13      giallo
14      giallo
15      giallo
16      giallo
17      giallo
18      giallo
19      giallo
20      giallo
21      giallo
22      giallo
23      giallo
24      giallo
25      giallo
26      giallo
27      giallo
28      giallo
29      giallo
30      giallo
31      giallo
32      giallo
33      giallo
34      giallo
35      giallo
36      giallo
37      giallo
38      giallo
39      giallo
40      giallo
41      giallo
42      giallo
43      giallo
44      giallo
45      giallo
46      giallo
47      giallo
48      giallo
49      giallo
50      giallo
51      giallo
52      giallo
53      giallo
54      giallo
55      giallo
56      giallo
57      giallo
58      giallo
59      giallo
60      giallo
61      giallo
62   

,Critico,Codice gruppo,Codice sottogruppo 1,Codice sottogruppo 2,Colore,Critico_0,Codice_sottogruppo_2_0,Criticality,Colore_low,Colour
0,X,02 AT,19 Interv. Rele' Diff.,NaN,Giallo,x,0,1,giallo,giallo
1,X,02 AT,19 Interv. Rele' Diff.,01 KD1 prolungato,Grigio,x,01 KD1 prolungato,1,giallo,giallo
2,X,02 AT,19 Interv. Rele' Diff.,02 KD1 ripetuto,Grigio,x,02 KD1 ripetuto,1,giallo,giallo
3,X,05 TCU,01 Avaria TCU,NaN,Giallo,x,0,1,giallo,giallo
4,X,05 TCU,01 Avaria TCU,01 Stotz 4F01,Grigio,x,01 Stotz 4F01,1,giallo,giallo
5,NaN,05 TCU,01 Avaria TCU,02 Life MVB TCU,Grigio,NaN,02 Life MVB TCU,0,giallo,giallo
6,X,05 TCU,03 Blocco Perm CA,NaN,Giallo,x,0,1,giallo,giallo
7,X,05 TCU,03 Blocco Perm CA,00 Blocco Perm CA,Grigio,x,00 Blocco Perm CA,1,giallo,giallo
8,NaN,05 TCU,03 Blocco Perm CA,01 Blocco Temp CA,Grigio,NaN,01 Blocco Temp CA,0,giallo,giallo
9,NaN,05 TCU,03 Blocco Perm CA,02 IncongruenzaK4,Grigio,NaN,02 IncongruenzaK4,0,giallo,giallo


In [24]:
# Modify "Codice gruppo" --> from "02 AT" to "2" and from "05 TCU" to "5":
cod_gruppo = []

for i in df["Codice gruppo"]:
    value = i.split(' ')[0]
    value = str(int(value))
    cod_gruppo.append(value)
    
df["Group_code"] = cod_gruppo
df["Group_code"]

0      2
1      2
2      2
3      5
4      5
5      5
6      5
7      5
8      5
9      5
10     5
11     5
12     5
13     5
14     5
15     5
16     5
17     5
18     5
19     5
20     5
21     5
22     5
23     5
24     5
25     5
26     5
27     5
28     5
29     5
30     5
31     5
32     5
33     5
34     5
35     5
36     5
37     5
38     5
39     5
40     5
41     5
42     5
43     5
44     5
45     5
46     5
47     5
48     5
49     5
50     5
51     5
52     5
53     5
54     5
55     5
56     5
57     5
58     5
59     5
60     5
61     5
62     5
63     5
64     5
65     5
66     5
67     5
68     5
69     5
70     5
71     5
72     5
73     5
74     5
75     5
76     5
77     5
78     5
79     5
80     5
81     5
82     5
83     5
84     5
85     5
86     5
87     5
88     5
89     5
90     5
91     5
92     5
93     5
94     5
95     5
96     5
97     5
98     5
99     5
100    5
101    5
102    5
103    5
104    5
105    5
106    5
107    5
108    5
109    5
110    5
1

In [25]:
# Modify "Codice sottogruppo 1" --> only the numbers (and without the 0 in front):
cod_sottogruppo_1 = []

for i in df["Codice sottogruppo  1"]:
    value = i.split(' ')[0]
    value = str(int(value))
    cod_sottogruppo_1.append(value)
    
df["Subgroup_code_1"] = cod_sottogruppo_1
df["Subgroup_code_1"]

0      19
1      19
2      19
3       1
4       1
5       1
6       3
7       3
8       3
9       3
10      3
11      3
12      3
13      3
14      3
15      3
16      3
17      3
18      3
19      3
20      3
21      3
22      3
23      3
24      3
25      3
26      3
27      3
28      3
29      3
30      3
31      3
32      3
33      3
34      3
35      3
36      3
37      3
38      3
39      3
40      5
41      5
42      5
43      5
44      5
45      5
46      5
47     10
48     10
49     10
50     11
51     11
52     11
53     11
54     11
55     11
56     11
57     11
58     11
59     11
60     11
61     11
62     14
63     14
64     15
65     15
66     16
67     16
68     16
69      2
70      4
71      6
72      7
73      7
74      7
75      7
76      7
77      8
78      9
79      9
80      9
81      9
82      9
83      9
84      9
85      9
86     12
87     12
88     13
89     13
90     13
91     13
92     13
93     17
94     31
95     31
96     31
97     31
98     31
99     31


In [26]:
# Keep only the final variables:
df2 = df[["Criticality","Colour","Group_code","Subgroup_code_1","Codice_sottogruppo_2_0"]]
df2

,Criticality,Colour,Group_code,Subgroup_code_1,Codice_sottogruppo_2_0
0,1,giallo,2,19,0
1,1,giallo,2,19,01 KD1 prolungato
2,1,giallo,2,19,02 KD1 ripetuto
3,1,giallo,5,1,0
4,1,giallo,5,1,01 Stotz 4F01
5,0,giallo,5,1,02 Life MVB TCU
6,1,giallo,5,3,0
7,1,giallo,5,3,00 Blocco Perm CA
8,0,giallo,5,3,01 Blocco Temp CA
9,0,giallo,5,3,02 IncongruenzaK4


In [27]:
# Modify "Codice sottogruppo 2" --> only the numbers (and without the 0 in front):
# issues:
#   1) there are NaN when the code is a pdm --> we transform them in 0 as in the dataset of the events recorded
#   2) there is the code "00" --> we arbitrary change it into "99""

# transform 0 in str in order to apply "split" to the entire column
df2.loc[df2["Codice_sottogruppo_2_0"] == 0, "Codice_sottogruppo_2_0"] = '0'
print(df2)

cod_sg_2 = []
for i in df2["Codice_sottogruppo_2_0"]:
    value = i.split(' ')[0]
    cod_sg_2.append(value)
print(cod_sg_2)

# identify index of 00:
indices = [index for index, value in enumerate(cod_sg_2) if value == "00"]
print(indices)  # = [7, 41, 73, 79]
# check:
print(df2.iloc[indices])

# change from 00 to 99:
for i in indices:
    cod_sg_2[i] = "99"
print(cod_sg_2)

# drop the 0 in front:
for i in range(len(cod_sg_2)):
    if cod_sg_2[i] != '0':
        cod_sg_2[i] = str(int(cod_sg_2[i]))
    else:
        cod_sg_2[i] == 0
print(cod_sg_2)

df2["Subgroup_code_2"] = cod_sg_2
df2

    Criticality   Colour Group_code Subgroup_code_1  Codice_sottogruppo_2_0
0             1   giallo          2              19                       0
1             1   giallo          2              19       01 KD1 prolungato
2             1   giallo          2              19         02 KD1 ripetuto
3             1   giallo          5               1                       0
4             1   giallo          5               1           01 Stotz 4F01
5             0   giallo          5               1         02 Life MVB TCU
6             1   giallo          5               3                       0
7             1   giallo          5               3       00 Blocco Perm CA
8             0   giallo          5               3       01 Blocco Temp CA
9             0   giallo          5               3       02 IncongruenzaK4
10            1   giallo          5               3          03 Stotz Pompa
11            1   giallo          5               3        04 Avaria Vent 1
12          

/tmp/ipykernel_2441328/1723027283.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Subgroup_code_2"] = cod_sg_2


,Criticality,Colour,Group_code,Subgroup_code_1,Codice_sottogruppo_2_0,Subgroup_code_2
0,1,giallo,2,19,0,0
1,1,giallo,2,19,01 KD1 prolungato,1
2,1,giallo,2,19,02 KD1 ripetuto,2
3,1,giallo,5,1,0,0
4,1,giallo,5,1,01 Stotz 4F01,1
5,0,giallo,5,1,02 Life MVB TCU,2
6,1,giallo,5,3,0,0
7,1,giallo,5,3,00 Blocco Perm CA,99
8,0,giallo,5,3,01 Blocco Temp CA,1
9,0,giallo,5,3,02 IncongruenzaK4,2


In [28]:
# Final dataset:
df3 = df2[["Criticality","Colour","Group_code","Subgroup_code_1","Subgroup_code_2"]]
print(df3)
df3.info()

# transform "Group_code","Subgroup_code_1","Subgroup_code_2" in integers:
df3['Group_code'] , df3['Subgroup_code_1'] , df3['Subgroup_code_2'] = df3['Group_code'].astype(int) , df3['Subgroup_code_1'].astype(int) , df3['Subgroup_code_2'].astype(int)
df3.info()
print(df3)

# merge the 3 codes:
code=[]
for i in range(len(df3)):
    if df3.loc[i, 'Subgroup_code_2'] == 0:
        codice = df3.loc[i,'Group_code'].astype(str) + '-' + df3.loc[i,'Subgroup_code_1'].astype(str)
        code.append(codice)
    else:
        codice = df3.loc[i,'Group_code'].astype(str) + '-' + df3.loc[i,'Subgroup_code_1'].astype(str) + '-' + df3.loc[i,'Subgroup_code_2'].astype(str)
        code.append(codice)
print(code)

df3["Code"] = code

finaldf = df3[["Criticality","Colour","Code"]]
print(finaldf)
finaldf.info()


# create a csv:
finaldf.to_csv('Codici_Diagnostici.csv', index=False)

    Criticality   Colour Group_code Subgroup_code_1 Subgroup_code_2
0             1   giallo          2              19               0
1             1   giallo          2              19               1
2             1   giallo          2              19               2
3             1   giallo          5               1               0
4             1   giallo          5               1               1
5             0   giallo          5               1               2
6             1   giallo          5               3               0
7             1   giallo          5               3              99
8             0   giallo          5               3               1
9             0   giallo          5               3               2
10            1   giallo          5               3               3
11            1   giallo          5               3               4
12            1   giallo          5               3               5
13            1   giallo          5             

/tmp/ipykernel_2441328/3128211231.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Group_code'] , df3['Subgroup_code_1'] , df3['Subgroup_code_2'] = df3['Group_code'].astype(int) , df3['Subgroup_code_1'].astype(int) , df3['Subgroup_code_2'].astype(int)
